# Cupy Part 01 - Cupy 기초
---

Cupy란 무엇일까요?

대부분의 사람들의 경우 Python에서 수학적인 계산을 할 때 Numpy를 사용할것이라 생각합니다.

Numpy는 훌륭하고, 빠르고, 편하고, 좋은 라이브러리입니다.

하지만 Cpu에서 돌아간다는 점 때문에, 직접 같은 수행을 만드는 것 보단 빠르지만, 대규모 작업을 처리할 때 조금 작업시간이 부담스러울 때도 있습니다.

Cupy란, Python에서 NVIDIA CUDA를 사용한 가속화 컴퓨팅을 제공하는 오픈소스 라이브러리입니다.

Cupy는 Numpy를 뛰어넘는 속도를 보여준다고합니다.

심지어, 자체 테스트에서 연산이 100배이상 차이나는 경우도 있었다고 합니다.

그렇다면 cupy는 어떻게 사용하는걸까요?

## Import
---

Cupy와 비교하기위해 Numpy도 같이 임포트 했습니다.

실제로 사용하실땐, Cupy의 매서드들은 모두 Numpy의 매서드와 동일하기 때문에, Numpy 부분을 Cupy로 대체하는 것만으로 대부분의 코드가 포팅가능합니다.

In [1]:
import cupy as cp
import numpy as np

## Simple Test Code
---

간단한 코드를 통해 사용법을 보겠습니다.

[1,2,3,4,5,6] 배열을 만들고
2 * 3으로 리쉐이프했습니다.

그 결과와 메모리 사용량을 Nvidia-Smi로 확인했습니다.

In [2]:
%%time

x_num = np.arange(6).reshape(2,3).astype("f")
print("X : ", x_num)
print("X sum : ", x_num.sum(axis=1))

!nvidia-smi

X :  [[0. 1. 2.]
 [3. 4. 5.]]
X sum :  [ 3. 12.]
Sun Aug  9 23:19:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1050    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P0    N/A /  N/A |    359MiB /  4042MiB |      9%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                           

In [3]:
%%time
x_cp = cp.arange(6).reshape(2,3).astype('f')
print("X : " , x_cp)
print("X sum : ", x_cp.sum(axis=1))

!nvidia-smi

X :  [[0. 1. 2.]
 [3. 4. 5.]]
X sum :  [ 3. 12.]
Sun Aug  9 23:19:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1050    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P0    N/A /  N/A |    435MiB /  4042MiB |     10%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                           

지난번에 보았던 설치 부분에서도, 봤던 코드와 비슷합니다.

GPU메모리 가장 아랫부분을 보면, 확실히 올라간 것이 보이죠?

GPU에서 정상 동작함을 알 수 있습니다.

## Time
---
그런데 한가지 이상한 점이 있습니다. Numpy보다 빠르다고 이야기를 했는데, Numpy가 Cupy보다 쉘 동작 시간이 더 짧음을 알 수 있습니다.

어째서 Numpy의 쉘 동작 시간이 더 빨랐을까요?

이는 GPU 솔루션을 잘 못 적용했기 때문입니다.

무슨 이야기냐, 다음과 같은 상황에서는 GPU보다 CPU가 더 빠른 성능을 낼 수도 있습니다.

  1. 계산량이 충분히 크지 않을경우
  2. 잘못된 구조로 GPU 아키텍쳐를 만들었을 경우
  3. 처음 로드하는 경우
 
지금의 경우에는 3번과 1번이 해당하겠네요.
 
자, 그러면 실제로도 연산량이 많을수록 속도 차이가 나는지 한번 코드의 연산량을 높여볼까요?

## Computing Time Test
---

지금 부터 간단하게 랜덤하게 생성한 NxN크기의 행렬을 두개 만든 후, 내적을 실행하도록 하겠습니다.

그리고 N의 크기를 증가시켜가면서 속도의 성능 차이가 얼마나 되는지도 함께 살펴보도록 하겠습니다.

### case 1. n=100

In [4]:
n = 100

In [5]:
%%time
a = np.random.rand(n,n)
b = np.random.rand(n,n)

result = np.matmul(a,b)

CPU times: user 15.3 ms, sys: 46.7 ms, total: 62 ms
Wall time: 17.3 ms


In [6]:
%%time
a = cp.random.rand(n,n)
b = cp.random.rand(n,n)

result = cp.matmul(a,b)

CPU times: user 151 ms, sys: 34.5 ms, total: 186 ms
Wall time: 185 ms


### case 2. n=1000

In [7]:
n = 1000

In [8]:
%%time
a = np.random.rand(n,n)
b = np.random.rand(n,n)

result = np.matmul(a,b)

CPU times: user 152 ms, sys: 86.1 ms, total: 238 ms
Wall time: 52.7 ms


In [9]:
%%time
a = cp.random.rand(n,n)
b = cp.random.rand(n,n)

result = cp.matmul(a,b)

CPU times: user 2.95 ms, sys: 2.2 ms, total: 5.15 ms
Wall time: 992 µs


### case 3. n=10000

In [10]:
n = 10000

In [11]:
%%time
a = np.random.rand(n,n)
b = np.random.rand(n,n)

result = np.matmul(a,b)

CPU times: user 2min 15s, sys: 14 s, total: 2min 29s
Wall time: 23.8 s


In [12]:
%%time
a = cp.random.rand(n,n)
b = cp.random.rand(n,n)

result = cp.matmul(a,b)

CPU times: user 98.8 ms, sys: 92.1 ms, total: 191 ms
Wall time: 48.5 ms


확실히 연산량이 늘어나면 늘어날 수록 Numpy에 비해 Cupy가 훨씬 빠른 속도를 내는게 보입니다.

반면 맨 처음에는 Numpy가 더 좋은 성능을 내고 있음을 알 수 있습니다.

## Cupy Data Type
---

처음 Cupy의 매서드와 Numpy의 매서드가 거의 동일하다고 말씀드렸습니다.

그러면 데이터 타입도 같을까요?(Numpy array등)

한번확인해보겠습니다.

In [13]:
Num_array = np.arange(6)
print(Num_array)
print(type(Num_array))

[0 1 2 3 4 5]
<class 'numpy.ndarray'>


In [14]:
Cupy_array = cp.arange(6)
print(Cupy_array)
print(type(Cupy_array))

[0 1 2 3 4 5]
<class 'cupy.core.core.ndarray'>


둘다 ndarray라는 이름이지만 Cupy는 cupy.core.core.이고, numpy는 numpy.ndarray입니다.

둘다 비슷하지만, Cupy는 cuda core에 올라가있습니다.

그러면 cupy ndarray를 numpy ndarray로 내릴땐 어떻게 해야할까요?

## Cupy.core.core.ndarray.get()
---

cupy의 ndarray에는 get()이라는 매서드가존재합니다.

get은 cuda core에 올라가있는 ndarray등을 numpy array로 내리는 역할을합니다.

이는 후에 데이터 시각화를 위해, numpy.ndarray등이 필요할때 많이 활용할 예정입니다.

In [15]:
cpu_array = Cupy_array.get()
print(cpu_array)
print(type(cpu_array))

[0 1 2 3 4 5]
<class 'numpy.ndarray'>


##  Cupy method
---

마지막으로 앞으로 자주쓰일 중요한 매서드를 몇가지 살펴보겠습니다.

Numpy 사용이 익숙하신 분들은 이 부분은 넘어가셔도 관계없습니다.

### cupy.array(내용)

Gpu에 cupy.ndarray형 선언용 method입니다.

In [16]:
x_gpu = cp.array([1, 2, 3])
print("{} : {}".format(x_gpu, type(x_gpu)))

[1 2 3] : <class 'cupy.core.core.ndarray'>


또한 Numpy Array를 Cupy Array로 바꿀 수 있습니다.

In [17]:
x_cpu = np.array([1,2,3])
print("{} : {}".format(x_cpu, type(x_cpu)))

x_gpu = cp.array(x_cpu)
print("{} : {}".format(x_gpu, type(x_gpu)))

[1 2 3] : <class 'numpy.ndarray'>
[1 2 3] : <class 'cupy.core.core.ndarray'>


### cupy.arange(strat, stop=None, step=1, dtype=None)
시작부터 끝지점까지 step 간격을 가진 cupyarray를 생성합니다.

In [18]:
cp.arange(1, 7, 2, float)

array([1., 3., 5.])

### cupy.empty(shape, dtype=\<class float>, order='C')
초기화한 cupy.array를 반환합니다.

In [19]:
cp.empty([2,3])

array([[0.34339195, 0.94128988, 0.81924949],
       [0.43385355, 0.70073458, 0.98856325]])

### cupy.ones(shape, dtype)
1로 초기화한 cupy.array를 반환합니다.

In [20]:
cp.ones([2,3])

array([[1., 1., 1.],
       [1., 1., 1.]])

### cupy.zeros(shape, dtype)
0으로 초기화한 cupy.array를 반환합니다.

In [21]:
cp.zeros([2,3])

array([[0., 0., 0.],
       [0., 0., 0.]])

### cupy.linalg.norm(cupy.ndarray)

Euclidean norm(a.k.a L2 norm)을 진행합니다.

In [22]:
l2_gpu = cp.linalg.norm(x_gpu)
print("{} : {}".format(l2_gpu, type(l2_gpu)))

3.7416573867739413 : <class 'cupy.core.core.ndarray'>


### cupy.cuda.Device(int).use()
cupy는 기본적으로 gpu 0을 사용하게 되어있습니다.
<br>이 명령어를 통해 원하는 gpu로 옮길 수 있습니다.
<br>저는 gpu가 여러대있는 컴퓨터가 없기때문에 실습을 생략하겠습니다.

### cupy.asnumpy(cupy.ndarray)
앞서 잠깐 살펴보았던 .get()메서드와 같은 역할입니다.
<br>둘 중 원하는 방법으로 사용하셔도 무방합니다.
<br>실습은 둘다 진행하겠습니다.

In [23]:
print("cupy.ndarray result : {}".format(cp.asnumpy(x_gpu)))
print(".get()       result : {}".format(x_gpu.get()))

cupy.ndarray result : [1 2 3]
.get()       result : [1 2 3]


### cupy.add(array1, array2)
두 어레이의 원소별 덧셈을 반환합니다.

In [24]:
x_gpu1 = cp.array([1,2,3])
x_gpu2 = cp.array([9,5,6])

cp.add(x_gpu1, x_gpu2)

array([10,  7,  9])

### cupy.subtract(array1, array2)
두 어레이의 원소별 뺄셈을 반환합니다.

In [25]:
cp.subtract(x_gpu1, x_gpu2)

array([-8, -3, -3])

### cupy.multiply(array1, array2)
두 어레이의 원소별 곱셈을 반환합니다.

In [26]:
cp.multiply(x_gpu1, x_gpu2)

array([ 9, 10, 18])

### cupy.divide(array1, array2)
두 어레이의 원소별 나눗셈을 반환합니다.

In [27]:
cp.divide(x_gpu1, x_gpu2)

array([0.11111111, 0.4       , 0.5       ])

### cupy.power(array1, array2)
두 어레이의 원소별 승곱을 반환합니다.

In [28]:
cp.power(x_gpu1, x_gpu2)

array([  1,  32, 729])

### cupy.mod(array1, array2)
두 어레이의 원소별 나머지를 반환합니다.

In [29]:
cp.mod(x_gpu1, x_gpu2)

array([1, 2, 3])

### cupy.absolute(array)
어레이의 원소별 절대값을 취한 값을 반환합니다.

In [30]:
x_minus = cp.array([-1, -2, -3])

cp.absolute(x_minus)

array([1, 2, 3])

### cupy.exp(array)
어레이의 원소별 Exponential 결과를 반환합니다.

In [31]:
cp.exp(x_gpu1)

array([ 2.71828183,  7.3890561 , 20.08553692])

### cupy.log(array)
어레이의 원소별 log를 수행합니다.

In [32]:
cp.log(x_gpu1)

array([0.        , 0.69314718, 1.09861229])

### cupy.sqrt(array)
어레이의 원소별 sqaure root 연산을 수행합니다.

In [33]:
cp.sqrt(x_gpu1)

array([1.        , 1.41421356, 1.73205081])

### cupy.square(array)
어레이의 원소별 제곱연산을 수행합니다.

In [34]:
cp.square(x_gpu1)

array([1, 4, 9])

### cupy.sin, cupy.cos, cupy.tan

다양한 삼각함수들도 지원하고 있습니다.

In [35]:
cp.sin(x_gpu1)

array([0.84147098, 0.90929743, 0.14112001])

In [36]:
cp.cos(x_gpu1)

array([ 0.54030231, -0.41614684, -0.9899925 ])

In [37]:
cp.tan(x_gpu1)

array([ 1.55740772, -2.18503986, -0.14254654])

### cupy.equal(array1, array2)
두 어레이의 각 원소별로 값이 같은지 비교하여 반환합니다.

In [38]:
x_gpu3 = cp.array([1, 4, 5])

cp.equal(x_gpu1, x_gpu3)

array([ True, False, False])

### cupy.maximum(array1, array2)
두 어레이의 각 원소별로 큰 값을 반환합니다.

In [39]:
x_gpu1 = cp.array([1, 4, 3])
x_gpu2 = cp.array([2, 3, 5])

cp.maximum(x_gpu1, x_gpu2)

array([2, 4, 5])

### cupy.minimum(array1, array2)
두 어레이의 각 원소별로 작은 값을 반환합니다.

In [40]:
cp.minimum(x_gpu1, x_gpu2)

array([1, 3, 3])

### cupy.floor(array)
원소별 floor연산을 수행 후 반환합니다.

In [41]:
x_float = cp.array([1.4, 1.5, 1.6])

cp.floor(x_float)

array([1., 1., 1.])

### cupy.ceil(array)
원소별 ceil연산을 수행 후 반환합니다.

In [42]:
cp.ceil(x_float)

array([2., 2., 2.])